In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt


from initial_states.utils.givensdecompose import _parity_gate
from qiskit import QuantumCircuit, QuantumRegister
from qiskit.circuit.library import XGate


### Drawing and verifying the parity gate implementing the FBS gate of Kerenidis and Prakash

In [ ]:
# n is total nb of qubits
n = 19
register = QuantumRegister(n)
linear_depth_parity_gate = False
reverse = False
a = 0
b = n-1
decomposition = _parity_gate(register, a, b, linear_depth_parity_gate, reverse)
circuit = QuantumCircuit(register)

planted_parity = 0 # even number of Xs
circuit.append(XGate(),[register[1]])
circuit.append(XGate(),[register[4]])
circuit.append(XGate(),[register[5]])
circuit.append(XGate(),[register[6]])

circuit.barrier(register)

for gate, qubits in decomposition:
    circuit.append(gate, qubits)
    
meas = qk.QuantumCircuit(n, n) # N qubits, N classical bits
meas.barrier(range(n)) # the barrier is optional, it is an instruction for the later transpiler
meas.measure(range(n), range(n)) # perform the measurement, record it in the classical bits
circuit.add_register(meas.cregs[0])
qc = circuit.compose(meas)
    
qc.draw(
    output="mpl", 
    filename="fig/example_circuit.pdf",
    scale = 0.9,
    fold=150)

checking that the parity of the qubit in (a,b) is stored in qubit a+1

In [ ]:
# Import Aer and set the backend simulator
from qiskit_aer import Aer
backend_sim = Aer.get_backend('qasm_simulator')
num_shots = 10
job_sim = backend_sim.run(
    qk.transpile(qc, backend_sim), 
    shots=num_shots
)
result_sim = job_sim.result()
counts = result_sim.get_counts(qc)

parity_qubit = 1
wrong_parity = False
for key in counts:
    measured_parity = int(str(key)[n-parity_qubit-1]) # need to reverse the order
    print('measured parity ', measured_parity)
    print(" ")
    if not measured_parity == planted_parity:
        print('wrong parity')
        wrong_parity = True
if  wrong_parity:
    print("test failed")
else:
    print('test passed')